# Query database

### Import packages, connect to the psql database

In [ ]:
# Import the relevant packages
import psycopg2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.time import Time
from astropy.io import ascii, fits
from astropy.coordinates import SkyCoord 
import astropy.units as u
from astroquery.irsa import Irsa

from penquins import Kowalski

from functions_misc import make_triplet, plot_triplet, get_cutouts
from functions_misc import get_dust_info, plot_lc
from functions_misc import get_xmatch_clu_glade, get_bgal_ebv


# Read the database secrets
info = ascii.read('./db_access.csv', format='csv')
info_db = info[info['db'] == 'db_kn_rt_user']
db_kn = f"host={info_db['host'][0]} dbname={info_db['dbname'][0]} \
port={info_db['port'][0]} user={info_db['user'][0]} \
password={info_db['password'][0]}"

# Connect to the database
con = psycopg2.connect(db_kn)
cur = con.cursor()
print(f"Connected to the '{info_db['dbname'][0]}' database")

# Read the secrets for kowalski access
secrets = ascii.read('secrets.csv', format='csv')
username_kowalski = secrets['kowalski_user'][0]
password_kowalski = secrets['kowalski_pwd'][0]


# Scoring

Assign or remove 'points' based on soft constraints

In [ ]:
# Start with full list, hard rejects will be removed HERE
scoring_df = pd.read_sql_query("SELECT name FROM candidate where hard_reject is NULL", con)

# Define the thresholds
thresh = {'rise': {'g': -1.0, 'r': -1., 'i': -0.5, 'sign_select': '<', 'sign_reject': '>'},
          'fade': {'g': 0.58, 'r': 0.43, 'i': 0.30, 'sign_select': '>', 'sign_reject': '<'}
          }

# Define the filters (list)
list_filters = ['g', 'r', 'i']

# Rise, fade, or both? (list, e.g. ['rise', 'fade'])
list_rise_fade = ['fade']

scores = {'rise_select': 5,
          'rise_pen': 0,
          'fade_select': 10,
          'fade_pen': -100,
         }

print(f"Working with {len(scoring_df)} candidates")
print(f"Considering the following filter(s): {list_filters}")
print("---")
print("Selected thresholds:")
for k1 in thresh.keys():
    for k2 in thresh[k1].keys():
        print(f"{k1} {k2}: {thresh[k1][k2]}")
print("---")
print("Scoring points:")
for k in scores.keys():
    print(f"{k}: {scores[k]}")

### Filter rise and fade (alerts, forced phot, stacked forced phot)

In [ ]:
# Filter for rise and fade for ALERTS
for rf in list_rise_fade:
    for f in list_filters:
        rf_filt = pd.read_sql_query(f"SELECT name FROM candidate WHERE index_{rf}_{f} {thresh[rf]['sign_select']} {thresh[rf][f]}",con).values
        print(f"{rf}_{f}_filt_alerts: {len(rf_filt)}" )

        # Assign points if condition is met, otherwise 0
        scoring_df[f'{rf}_{f}_filt_alerts'] = [scores[f"{rf}_select"] if name in rf_filt else 0 for name in scoring_df['name']]


# Filter for rise and fade for FORCED PHOTOMETRY
for rf in list_rise_fade:
    for f in list_filters:
        rf_filt = pd.read_sql_query(f"SELECT name FROM candidate WHERE index_{rf}_forced_{f} {thresh[rf]['sign_select']} {thresh[rf][f]}",con).values
        print(f"{rf}_{f}_filt_forced: {len(rf_filt)}" )

        # Assign points if condition is met, otherwise 0
        scoring_df[f'{rf}_{f}_filt_forced'] = [scores[f"{rf}_select"] if name in rf_filt else 0 for name in scoring_df['name']]


# Filter for rise and fade for STACKED FORCED PHOTOMETRY
for rf in list_rise_fade:
    for f in list_filters:
        rf_filt = pd.read_sql_query(f"SELECT name FROM candidate WHERE index_{rf}_stack_{f} {thresh[rf]['sign_select']} {thresh[rf][f]}",con).values
        print(f"{rf}_{f}_filt_stack: {len(rf_filt)}" )

        # Assign points if condition is met, otherwise 0
        scoring_df[f'{rf}_{f}_filt_stack'] = [scores[f"{rf}_select"] if name in rf_filt else 0 for name in scoring_df['name']]

### Penalize slow rise or fade (alerts, forced phot, stacked forced phot)

In [ ]:
# Penalize slow rise and fade for ALERTS
for rf in list_rise_fade:
    for f in list_filters:
        rf_filt = pd.read_sql_query(f"SELECT name FROM candidate WHERE index_{rf}_{f} {thresh[rf]['sign_reject']} {thresh[rf][f]}",con).values
        print(f"{rf}_{f}_pen_alerts: {len(rf_filt)}" )

        # Assign points if condition is met, otherwise 0
        scoring_df[f'{rf}_{f}_pen_alerts'] = [scores[f"{rf}_pen"] if name in rf_filt else 0 for name in scoring_df['name']]
        

# Penalize slow rise and fade for FORCED PHOTOMETRY
for rf in list_rise_fade:
    for f in list_filters:
        rf_filt = pd.read_sql_query(f"SELECT name FROM candidate WHERE index_{rf}_forced_{f} {thresh[rf]['sign_reject']} {thresh[rf][f]}",con).values
        print(f"{rf}_{f}_pen_forced: {len(rf_filt)}" )

        # Assign points if condition is met, otherwise 0
        scoring_df[f'{rf}_{f}_pen_forced'] = [scores[f"{rf}_pen"] if name in rf_filt else 0 for name in scoring_df['name']]


# Penalize slow rise and fade for STACKED FORCED PHOTOMETRY
for rf in list_rise_fade:
    for f in list_filters:
        rf_filt = pd.read_sql_query(f"SELECT name FROM candidate WHERE index_{rf}_stack_{f} {thresh[rf]['sign_reject']} {thresh[rf][f]}",con).values
        print(f"{rf}_{f}_pen_stack: {len(rf_filt)}" )

        # Assign points if condition is met, otherwise 0
        scoring_df[f'{rf}_{f}_pen_stack'] = [scores[f"{rf}_pen"] if name in rf_filt else 0 for name in scoring_df['name']]

## Penalize candidates with long duration in forced photometry

In [ ]:
# Penalize long duration transients - TOTAL
duration_pen = pd.read_sql_query("SELECT name FROM candidate \
WHERE duration_tot > 14", con).drop_duplicates('name').values
print('duration_pen: ' + str(len(duration_pen)))

scoring_df['duration_pen'] = [-100 if name in duration_pen else 0 for name in scoring_df['name']]

# Penalize long duration transients - INDIVIDUAL FILTERS
duration_dict = {"g": 10, "r": 12, "i": 14}
for f in ["g", "r", "i"]:
    duration_pen = pd.read_sql_query(f"SELECT name FROM candidate \
    WHERE duration_{f} > {duration_dict[f]}", con).drop_duplicates('name').values
    print(f'duration_pen_{f}: ' + str(len(duration_pen)))
    scoring_df[f'duration_pen_{f}'] = [-100 if name in duration_pen else 0 for name in scoring_df['name']]

##  Crossmatch scoring
Have ranges in assigning/penalizing points based on rising rate -> fast +1, slow-1, between 0. Ranges defined by RCF SN results, and/or 2018cow

Points for candidates with more than 6 detections

Point there is a non-PSF LS source with phot_z less than x

Penalize if PS object within 1.5 arcsec

Point is PS source with sgscore less than x and mag brighter than y

### Present in either CLU or GLADE 
NOTE: GLADE crossmatch is not yet implemented, WIP.

In [ ]:
# Filter for match in either CLU or GLADE 
galaxy_match_filt = pd.read_sql_query("SELECT name FROM crossmatch \
WHERE (clu_dist_kpc < 100) and (clu_distmpc > 10)", con).drop_duplicates('name').values
print('galaxy_match_filt: ' + str(len(galaxy_match_filt)))


# Now the CLU crossmatching is giving zero points, change as desired. 
scoring_df['galaxy_match_filt'] = [1 if name in galaxy_match_filt else 0 for name in scoring_df['name']]

# AGN scores

In [ ]:
from functions_misc import agn_b_scores

agn_wise =[]
b_score = []

# # print('Quering II/328/allwise \n## w1-2 0.6..1.7 w2-w3 2.2 .. 4.5')
# for name in scoring_df['name']:
#     agn,b = agn_b_scores(name,username_kowalski,password_kowalski)
#     agn_wise.append(agn)
#     b_score.append(b)

# scoring_df['agn_wise'] = agn_wise
# scoring_df['gal_latitude'] = b_score

# Results

In [ ]:
# Create dataframe with the results
result_df = pd.DataFrame([])
result_df['name'] = scoring_df['name']
result_df['sum'] = scoring_df.sum(axis=1)
result_df.sort_values(by='sum', ascending=False)[:12]

### Distribution of the resultng scores

In [ ]:
# Define a scoring threshold
score_thresh = 1

fig, ax = plt.subplots(1,1)
print(f"There are {len(result_df[result_df['sum'] > score_thresh])} candidates above the scoring threshold")

bins = 'auto'
#bins = np.arange(np.min(result_df['sum']), np.max(result_df['sum']), 0.5)
bins = np.arange(-10, np.max(result_df['sum'])+1, 0.5)

ax.hist(result_df['sum'], bins=bins)
ax.set_yscale('log')
ax.set_xlabel("Score")
#plt.savefig("score_distribution.png")
plt.show()


# Plot up the results
Define the list of candidates to plot. By default, it plots all those with score larger than the `score_thresh` previously defined

In [ ]:
# Define a recency threshold (in days)
recency_thresh = 21

In [ ]:
# Plotting cell…
list_names = result_df.sort_values(by='sum', ascending=False)[result_df['sum'] > score_thresh]['name']

# Select only recent stuff
list_recent = pd.read_sql_query(f"SELECT name FROM lightcurve WHERE jd > {Time.now().jd - recency_thresh}", con).drop_duplicates('name')
list_names = list(n for n in list_names if n in list(list_recent['name']))

# Get CLU and GLADE crossmatch information
clu, glade = get_xmatch_clu_glade(list_names, con, cur)

# Get coords, Galactic latitude and E(B-V)
bgal_ebv = get_bgal_ebv(list_names, con, cur)

list_out = []

for name in list_names:
    print(name)
    clu_crossmatch = clu[clu['name'] == name]
    if clu_crossmatch.empty:
        print("No CLU crossmatch")
    else:
        print("CLU crossmatch:")
        print(clu[clu['name'] == name])
    print(f"Coordinates: RA, Dec = {'{:.6f}'.format(float(bgal_ebv[bgal_ebv['name'] == name]['ra']))}, {'{:.5f}'.format(float(bgal_ebv[bgal_ebv['name'] == name]['dec']))}")
    print(f"Extinction: E(B-V) = {'{:.2f}'.format(float(bgal_ebv[bgal_ebv['name'] == name]['ebv']))}")
    print(f"Galactic latitude: b_Gal = {'{:.2f}'.format(float(bgal_ebv[bgal_ebv['name'] == name]['b_gal']))}")
        
    ###print(glade[glade['name'] == name])
    alerts = get_cutouts(name, username_kowalski, password_kowalski)
    triplet = make_triplet(alerts[0])
    plot_triplet(triplet)
    print(f"Alerts light curve for {name}")
    plot_lc(name, con, cur, forced=False, stack=False, plot_alerts=True, save=False, inset=False, tr=triplet, plot_cow=False, plot_gfo=False, plot_bulla=False, filtermatch='g')    
    print(f"Forced photometry light curve for {name}")
    plot_lc(name, con, cur, forced=True, stack=False, plot_alerts=True, save=False, inset=False, tr=triplet, plot_cow=False, plot_gfo=False, plot_bulla=False, filtermatch='g')
    print(f"Stacked forced photometry light curve for {name}")
    plot_lc(name, con, cur, forced=True, stack=True, plot_alerts=True, save=False, inset=False, tr=triplet, plot_cow=False, plot_gfo=False, plot_bulla=False, filtermatch='g')
    print("------")
    list_out.append(name)
print(f"Found {len(list_out)} candidates")
print(list_out)

****
****
# The End
What you actually need for the scanning finishes here.
****
****

## Tables
There are multiple tables in the database. Here you can print the names of the tables and the names of the columns available in each table

In [ ]:
# Table names
tables = pd.read_sql_query("SELECT table_name FROM information_schema.tables WHERE table_schema='public' ",con)

for t in tables["table_name"]:
    print(f"Table {t}:")
    q = pd.read_sql_query(f"SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{t}'", con)
    print(q)

## Close the database connection

In [ ]:
cur.close()
con.close()